# Practical 1: Processing ground-based gravity data

In this practical, we'll process a set of synthetic gravity surveys near the city of Cape Town, South Africa. The surveys were conducted with a LaCoste & Romberg Model G gravimeter and tied to a single base station (station 0).

## Data 

For this practical, you'll need:

* A zip archive with the survey data: [`cape-town-surveys.zip`](https://github.com/leouieda/gravity-processing/raw/main/data/cape-town-surveys.zip)
* The gravimeter calibration table to convert readings to mGal: [`gravimeter-scaling.csv`](https://raw.githubusercontent.com/leouieda/gravity-processing/main/data/gravimeter-scaling.csv)

Download both files, unzip the archive, and place everything into the **same folder as this notebook**. Your should folder structure should look like this:

```
gravity-practicals/
├── cape-town-surveys/
│   ├── cape-town-gravity-day-1.csv
│   ├── cape-town-gravity-day-2.csv
│   ├── cape-town-gravity-day-3.csv
│   ├── cape-town-gravity-day-4.csv
│   └── cape-town-gravity-day-5.csv
├── gravimeter-scaling.csv
└── practical1.ipynb
```

### Survey data

The survey files are in Comma Separated Values (CSV) format, which is a table of values with rows representing individual survey points and columns representing the position, time, and meter reading. For example, these are the first few lines of one of these files:

```
station_id,longitude,latitude,easting,northing,elevation,time_minutes,reading
0,18.34444,-34.12971,255105.43,6220276.33,32.2,0.0,2555.08
1,18.37418,-34.19583,258037.64,6213013.01,18.4,8.0,2565.12
2,18.40388,-34.23972,260899.25,6208214.71,25.0,14.0,2569.49
3,18.41112,-34.16444,261353.99,6216582.03,228.7,23.0,2514.59
```

* Station ID is a unique identifier (across all surveys) for each survey point. 
* Latitude and longitude are geographic coordinates in degress referenced to the WGS84 ellipsoid.
* Easting and northing are UTM coordinates in meters.
* Elevation is the height of gravimeter above the WGS84 ellipsoid in meters.
* Time are the minutes since the first reading at the base station.
* Reading is the gravimeter reading (unscaled) at that stattion.

The surveys were conducted in a loop, beginning and ending at the base station.

### Gravimeter calibration table

The gravimeter calibration table is particular to the gravimeter used for this survey. The table was provided by the manufacturer. This is what the first few lines of the table look like:

```
counter_reading,value_mgal,interval_factor
0000,0000.00,1.00636
0100,0100.64,1.00621
0200,0201.26,1.00609
0300,0301.87,1.00597
0400,0402.46,1.00588
0500,0503.05,1.00579
0600,0603.63,1.00570
0700,0704.20,1.00563
```

Convert the reading to mGal using the table values and the following equation:

```
reading_mgal = (reading - counter_reading) * interval_factor + value_mgal
```

### Base station

The base station for all surveys was the same. Here are it's properties:

* Station ID: 0
* Absolute gravity: 979656.12 mGal